# Data Exploration Outside of Main Notebook

This notebook is a bit more messy and exploratory than the main notebook, but I'm going to do some data exploration here to iterate faster.

In [5]:
import os
import re
import torch
import gdown
import jsonlines
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
from transformers import GPT2Tokenizer, GPT2TokenizerFast, AutoTokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
import ftfy
from lm_dataformat import Reader
from gpt_generate import gpt_generate, create_prompt_txt_from_df, print_ground_truth_from_df
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import torch

torch.__version__

'1.12.0+cu113'

### Setting Up Directories

In [6]:
import os
os.makedirs("prompts/contexts", exist_ok=True)
os.makedirs("prompts/questions", exist_ok=True)
os.makedirs("prompts/answers", exist_ok=True)
os.makedirs("prompts/task_description", exist_ok=True)
os.makedirs("prompts/ground_truths", exist_ok=True)
os.makedirs("prompts/prompts_with_relevance", exist_ok=True)
os.makedirs("prompts/prompts_without_relevance", exist_ok=True)

# Preparing the Initial Dataset

To create some initial prompts for testing, I went on LessWrong.org and read some of the prompts from the comment section on [this post](https://www.lesswrong.com/posts/8c8AZq5hgifmnHKSN/agi-safety-faq-all-dumb-questions-allowed-thread#comments). I also created a few with the help of the [Natural Questions dataset from Google](https://ai.google.com/research/NaturalQuestions/visualization) and created a few by hand. To make things faster, I stored the data in Google Sheets and then exported it to CSV.

For quick iteration, I used GPT-2, GPT-J, GPT-3, and instruct-GPT-3 to get a feel for model performance. For the difficult examples from the dataset, all models performed poorly. However, as I added more few-shot examples and better context engineering, the models started to perform better (though still not great for the smaller models). This notebook will show these observations in a quantitative way while still giving my qualitative observations.

Here's what the data looks like (ignore the columns past explanation, I'll only use them post-training if I do):

In [8]:
df = pd.read_csv("data/qa-relevance-dataset.csv")
print(len(df))
df.head(2)

17


,question,answer,relevance,explanation,difficulty,deceptive,improved_question,improved_answer
0,"When I‘m on lesswrong, I often read posts that have the assumption that an AGI can be described as a utility maximizer. Why is that?",An AGI that was not a utility maximizer would make more progress towards whatever goals it had if it modified itself to become a utility maximizer.,relevant,it explains an AGI is described as a utility maximizer because that's how the AGI would make more progress towards its goal.,medium,no,NaN,NaN
1,"When I‘m on lesswrong, I often read posts that have the assumption that an AGI can be described as a utility maximizer. Why is that?",I jumped in the river to save the little boy.,not relevant,"it is talking about jumping in a river to save a boy, but the question is about AGI.",NaN,NaN,NaN,NaN


Let's list out all of the benchmark examples in full.

In [15]:
for i in range(len(df)):
    print(f"Ground truth {i}")
    print_ground_truth_from_df(df, i, f"prompts/ground_truths/ground_truth_{i}.txt")
    print("----------------------------------------------------")

Ground truth 0
Users on a website ask questions and then other users answer them. Some users write answers that are not relevant to the question. The goal is to explain why the answer is relevant or not relevant to the question.

QUESTION: When I‘m on lesswrong, I often read posts that have the assumption that an AGI can be described as a utility maximizer. Why is that?

ANSWER: An AGI that was not a utility maximizer would make more progress towards whatever goals it had if it modified itself to become a utility maximizer.

Below contains an explanantion of why the answer above is or isn't relevant to the question:

This answer is relevant because it explains an AGI is described as a utility maximizer because that's how the AGI would make more progress towards its goal.
----------------------------------------------------
Ground truth 1
Users on a website ask questions and then other users answer them. Some users write answers that are not relevant to the question. The goal is to expl